## Branching Point Detector
Given a backbone model specified in the format of ```*.prototxt```, the branching point detector will automatically divide it into sequential blocks.

In [ ]:
from main.auto_models import MTSeqBackbone

In [ ]:
prototxt = 'models/deeplab_resnet34_adashare.prototxt' # MobileNetV2: models/mobilenetv2.prototxt
backbone = MTSeqBackbone(prototxt)
B = len(backbone.basic_blocks)
print('The number of blocks is {}.'.format(B))

The user can __further specify coarse-grained branching points__ based on the auto-generated branching points by defining a mapping dictionary.
The following example maps the 32 branching points to 5 coarser ones:

| coarse |    0 |   1 |     2   |   3   |  4 |
|:------:|:------:|:---------:|:---------:|:----:|:--:|
|  fined | 0 | 1,2,3 | 4,5,6,7 | 8,9,10,11,12,13 | 14,15,16 |

__Note:__ The ```mapping``` dict has the last element ```5:[17]``` to indicate the all-shared models. In other words, if you're mapping $M$ branching points to $N$ coarser ones, ```mapping``` will contains $N+1$ elements in which the last one is ```N:[M]```.

In [ ]:
coarse_B = 5
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}
# mapping = {0:[0,1,2,3,4,5,6], 1:[7,8,9,10,11,12,13,14,15,16,17], 2:[18,19,20,21,22], 
#            3:[23,24,25,26,27,28,29,30], 4:[31], 5:[32]} # mapping for MobileNetV2

## Design Spce Enumerator
Given the number of tasks $T$ and the number of branching points $B$, the design space enumerator could explore the tree-structured multi-task model architectures space completely.

In [ ]:
from main.algorithms import enumerator 

In [ ]:
T = 3 # NYUv2 has 3 tasks, Taskonomy has 5 tasks
layout_list = enumerator(T, coarse_B)
print('There are {} layouts in the design space.'.format(len(layout_list)))

## Task Accuracy Estimator
For each layout in the design space, we will estimate its task accuracy from the task accuacy of associated 2-task models.

In [ ]:
from main.algorithms import reorg_two_task_results, compute_weights, metric_inference

### Step 1: Load 2-task models results

__The task accuracy of all the 2-task models should be stored in excel and organized as the following example.__

* Each column represents different 2-task combinations. 
For $(a,b)-i: i \in \{0,1\}$, $(a,b)$ refers to the 2-task model of task $a$ and task $b$, and $-i$ means the current column is the accuracy of $i$-th task -- $0$ is task $a$, $1$ is task $b$.

* Each row represents the branching points of the 2-task models.
Notice that $0$ means independent models, while $B$ means all-shared models.

More examples can be found in the folder ```2task/*.xlsx```.

In [ ]:
import pandas as pd
two_task_pd = pd.read_excel('2task/NYUv2_2task_metrics_resnet_1129_val_acc.xlsx',engine='openpyxl',index_col=0)
two_task_pd

### Step 2: Compute score weights

The 2-task results will be reorganized by ```reorg_two_task_results``` for the further computation, and the task weights for the layout scores are computed by ```compute_weights```.

In [ ]:
two_task_metrics = reorg_two_task_results(two_task_pd, T, coarse_B)
score_weights = compute_weights(two_task_pd, T)

### Step 3: Compute layout score from accociated 2-task models

For each layout in the design space ```layout_list```, figure out the accociated 2-task models by ```metric_inference```, then set the final score by ```L.set_score_weighted```.

In [ ]:
# Run for all L
for L in layout_list:
    print('Layout: {}'.format(L))
    
    subtree = metric_inference(L, two_task_metrics)
    print('Associated 2-task models for each task: {}'.format(subtree))
    
    L.set_score_weighted(score_weights)
    print('Final Score: {:.4f}'.format(L.score))
    
    print('=' * 100)

### Step 4: Sort the layouts

In [ ]:
layout_order = sorted(range(len(layout_list)), key=lambda k: layout_list[k].score,reverse=True)

In [ ]:
for i in range(0,len(layout_order)):
    print('Layout Idx: {}'.format(layout_order[i]))
    L = layout_list[layout_order[i]]
    print('Layout: {}'.format(L))
    print('Final Score: {:.4f}'.format(L.score))
    print('=' * 100)

## Appendix 1: Dataloader, Loss, and Metrics

We provide dataloader, loss functions, and metrics evaluations for NYUv2 and Taskonomy.

In [ ]:
from torch.utils.data import DataLoader

from data.nyuv2_dataloader_adashare import NYU_v2
from data.taskonomy_dataloader_adashare import Taskonomy
from data.pixel2pixel_loss import NYUCriterions, TaskonomyCriterions
from data.pixel2pixel_metrics import NYUMetrics, TaskonomyMetrics

In [ ]:
dataroot = 'data/NYUv2' # Your root

criterionDict = {}
metricDict = {}

### NYUv2

In [ ]:
tasks = ['segment_semantic','normal','depth_zbuffer']
cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

dataset = NYU_v2(dataroot, 'train', crop_h=321, crop_w=321)
trainDataloader = DataLoader(dataset, 16, shuffle=True)

dataset = NYU_v2(dataroot, 'test', crop_h=321, crop_w=321)
valDataloader = DataLoader(dataset, 16, shuffle=True)

for task in tasks:
    criterionDict[task] = NYUCriterions(task)
    metricDict[task] = NYUMetrics(task)

### Taskonomy

In [ ]:
tasks = ['segment_semantic','normal','depth_zbuffer','keypoints2d','edge_texture']
cls_num = {'segment_semantic': 17, 'normal': 3, 'depth_zbuffer': 1, 'keypoints2d': 1, 'edge_texture': 1}
    
dataset = Taskonomy(dataroot, 'train', crop_h=224, crop_w=224)
trainDataloader = DataLoader(dataset, batch_size=16, shuffle=True)

dataset = Taskonomy(dataroot, 'test_small', crop_h=224, crop_w=224)
valDataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for task in tasks:
    criterionDict[task] = TaskonomyCriterions(task, dataroot)
    metricDict[task] = TaskonomyMetrics(task, dataroot)

## Appendix 2: 2-Task Models & N-Task Models
We need to train all the 2-task models at different branching points for the performance table, and the n-task models we select after estimating and sorting their task accuracy. Therefore we also provide __a model generator that can automatically build up the 2-task models based on the given branching points, and the n-task models based on the given layout__.

In [ ]:
import torch
from main.auto_models import MTSeqModel
from main.algorithms import coarse_to_fined

### 2-Task Model
* Inputs: prototxt, the branching point, the number of branching points, the feature dimension and the number of class for task heads
* __Note:__
    * Given a coarse branch point, we can convert it to a fined branch point from the mapping.
    * The feature dimension can be defined by the user or derived from the backbone model.
    * Remember to select 2 tasks from the task set.

In [ ]:
coarse_branch = 3
fined_branch = mapping[coarse_branch][0]
feature_dim = backbone(torch.rand(1,3,224,224)).shape[1]

In [ ]:
two_task = ['segment_semantic','normal'] # Select two tasks as you like
two_cls_num = {task: cls_num[task] for task in two_task}

In [ ]:
model = MTSeqModel(prototxt, branch=fined_branch, fined_B=B, feature_dim=feature_dim, cls_num=two_cls_num)

### N-Task Model
* Inputs: prototxt, a layout, the feature dimension and the number of class for task heads
* __Note:__
    * Given a layout enumerated under the coarse branching points, we can use ```coarse_to_fined``` to convert it to a layout under the fined branching points

In [ ]:
coarse_layout = layout_list[45]
fined_layout = coarse_to_fined(coarse_layout, B, mapping)

In [ ]:
model = MTSeqModel(prototxt, layout=fined_layout, feature_dim=feature_dim, cls_num=cls_num)

## Appendix 3: Trainer Functions

We further provide trainer functions to train the 2-task and n-task models.

In [ ]:
from main.trainer import Trainer

In [ ]:
trainer = Trainer(model.cuda(), tasks, trainDataloader, valDataloader, criterionDict, metricDict)
trainer.train(20000)